In [3]:
import requests

from bs4 import BeautifulSoup

def download_page(url: str) -> str:
    response = requests.get(url)
    response.raise_for_status()
    return response.text

In [ ]:
# Test download_page
# page_link = input("Input link to results page: ")
# response = download_page(page_link) 

In [5]:
def parser(response: str) -> dict:
    soup = BeautifulSoup(response, 'html.parser')
    result = {}
    for row in soup.table.find_all('tr'):
        row_header = row.th.get_text()
        row_cell = row.td.get_text()
        result[row_header] = row_cell
        return result

In [6]:
# Test parser
# result = parser(response)
# for r in result:
#     print(r)

In [7]:
def split_data(result: dict) -> dict:
    for r in result:
        splitted = r.split("\n")
        header = splitted[0].split(" ")
        body = splitted[1:]
        data= []
        for i in range(0, len(body) - 1, 3):
            
            # Extract and calc distance val
            p1 = int(body[i][body[i].find("(") + 1: body[i].find("%")])
            p2 = int(body[i+1][body[i+1].find("(") + 1: body[i+1].find("%")])
            pA = (p1 + p2) / 2
            distance = round(1 - (pA/100), 2)
            data.append({header[0]: body[i], header[1]: body[i+1], header[2]: body[i+2], "Percentage1": p1, "Percentage2": p2, "Average Percentage": pA, "Distance": distance})
            
        return data
    


In [8]:
# Test split_data
# data = split_data(result)

In [9]:
# Write formatted info from cite into csv

import csv

def write_data(data: dict, filename: str):
    with open(filename, "w", newline="") as csvfile:
        fieldnames = list(data[0].keys())
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        writer.writeheader()
        writer.writerows(data)

In [10]:
# Test write_data
# write_data(data, "extractMoss.csv") #any csv file

In [11]:
# To use: input site addr & csv filename to write to
# Output: Csv file in same folder with extracted info & calculated distance

def extractInfo(siteAddr: str, writeFile: str):
    response = download_page(siteAddr)
    result = parser(response)
    data = split_data(result)
    write_data(data, writeFile)
    

In [12]:
# RUN THIS CELL TO USE

siteAddr = input("Input address of site to extract data: ")
writeFile = input("Input csv filename (include .csv) to write data to: ")

extractInfo(siteAddr, writeFile)

Input address of site to extract data: http://moss.stanford.edu/results/7/6164340468562/
Input csv filename (include .csv) to write data to: test.csv
